In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
from file_cache import *
from fastai.vision import *
from fastai.callbacks.hooks import *

gc.collect()
def unet_learner( arch:Callable, pretrained:bool=True, blur_final:bool=True,
                 norm_type:Optional[NormType]=NormType, split_on:Optional[SplitFuncOrIdxList]=None, blur:bool=False,
                 self_attention:bool=False, y_range:Optional[Tuple[float,float]]=None, last_cross:bool=True,
                 bottle:bool=False, cut:Union[int,Callable]=None,
                 n_classes=2, img_size = (224,224),
                 **learn_kwargs:Any)->Learner:
    "Build Unet learner from `data` and `arch`."
    "blur: do maxpolling or not"
    body = create_body(arch, pretrained, cut)
    model = to_device(models.unet.DynamicUnet(body, n_classes=n_classes, img_size=img_size, blur=blur, blur_final=blur_final,
          self_attention=self_attention, y_range=y_range, norm_type=norm_type, last_cross=last_cross,
          bottle=bottle), 'cuda')
    return model

# model = unet_learner( models.resnet50, n_classes=2, img_size = (224,224) )

# model

2020-05-22 18:41:41,519 util_log.py[153] INFO Start the program at:amax7, 127.0.1.1, with:Load module


File_cache: Adjust notebook work fold to:/share/felix/pj/brain_seg/


In [2]:
# model(torch.rand(10,3,224,224).cuda()).shape

In [3]:
from torchvision import transforms
class DataSet_lung(Dataset):

    def __init__(self, ds_type='train'):
        super(DataSet_lung, self).__init__()
        self.image_size = 256
        df = self.get_df()
        #print(df.valid.value_counts())
        if ds_type == 'train':
            #print(ds_type)
            self.df = df.loc[df.valid == False]
        elif ds_type == 'valid':
            self.df = df.loc[df.valid == True]
        
        self.transforms = transforms.Compose([transforms.ToTensor(),
                              transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                             ])

            
    @lru_cache()
    def get_df(self):
        root = '/home/felix/pj/lung_detectron/data/lung/unet_v9'
        file_list = glob(f'{root}/images/*.*')
        df = pd.DataFrame({'img':file_list})
        df['valid'] = False
        total = len(df)
        df['valid'].iloc[:total//5] = True
        print(df.valid.value_counts())
        def get_label_file(file):
            file = file.replace('.png', '_P.png')
            file = file.replace('images', 'labels')
            return file
        df['label'] = df.img.apply(lambda val: get_label_file(val))
        return df.sample(frac=0.05)

    def __getitem__(self, index):
        img = cv2.imread(self.df.img.iloc[index])
        img = cv2.resize(img,(224, 224))
        
        label = cv2.imread(self.df.label.iloc[index])[:,:,0]
        label =  cv2.resize(label, (224,224)).astype(int)
        #label = np.stack([ np.where(label==i, 1, 0) for i in range(2)])
        return  self.transforms(img ) ,label

    def __len__(self):
        return len(self.df)

for sn, (a, b) in enumerate(DataSet_lung()):
    print(a.shape, b.shape, np.unique(b))
    if sn> 5: break


False    6334
True     1583
Name: valid, dtype: int64
torch.Size([3, 224, 224]) (224, 224) [0 1]
torch.Size([3, 224, 224]) (224, 224) [0 1]
torch.Size([3, 224, 224]) (224, 224) [0 1]
torch.Size([3, 224, 224]) (224, 224) [0 1]
torch.Size([3, 224, 224]) (224, 224) [0 1]
torch.Size([3, 224, 224]) (224, 224) [0 1]
torch.Size([3, 224, 224]) (224, 224) [0 1]


In [4]:
import os

import torch
from torch.nn import functional as F
from torch.utils.data import DataLoader
from torchvision.datasets import MNIST
from torchvision import transforms
import pytorch_lightning as pl

class BrainModel(pl.LightningModule):

    def __init__(self):
        super(BrainModel, self).__init__()
        self.unet =  unet_learner( models.resnet50, n_classes=2, img_size = (224,224) ) 

    def forward(self, x):
        # called with self(x)
        #print(x.shape)
        return self.unet(x)

    def training_step(self, batch, batch_nb):
        # REQUIRED
        x, y = batch
        y_hat = self(x)
        loss = F.cross_entropy(y_hat, y)
        
        #print(y_hat.shape, y.shape, loss)
        tensorboard_logs = {'train_loss': loss}
        return {'loss': loss, 'log': tensorboard_logs}

    def validation_step(self, batch, batch_nb):
        # OPTIONAL
        x, y = batch
        y_hat = self(x)
        return {'val_loss': F.cross_entropy(y_hat, y)}

    def validation_epoch_end(self, outputs):
        # OPTIONAL
        avg_loss = torch.stack([x['val_loss'] for x in outputs]).mean()
        tensorboard_logs = {'val_loss': avg_loss}
        print({'val_loss': avg_loss})
        return {'val_loss': avg_loss, 'log': tensorboard_logs}

    def test_step(self, batch, batch_nb):
        # OPTIONAL
        print('test_step')
        x, y = batch
        y_hat = self(x)
        return {'test_loss': F.cross_entropy(y_hat, y)}

    def test_epoch_end(self, outputs):
        # OPTIONAL
        print('test_epoch_end')
        avg_loss = torch.stack([x['test_loss'] for x in outputs]).mean()
        logs = {'test_loss': avg_loss}
        return {'test_loss': avg_loss, 'log': logs, 'progress_bar': logs}

    def configure_optimizers(self):
        # REQUIRED
        # can return multiple optimizers and learning_rate schedulers
        # (LBFGS it is automatically supported, no need for closure function)
        return torch.optim.Adam(self.parameters(), lr=0.0002)

    def train_dataloader(self):
        # REQUIRED
        return DataLoader(DataSet_lung('train'), batch_size=8)
        #return DataLoader(MNIST(os.getcwd(), train=True, download=True, transform=transforms.ToTensor()), batch_size=32)

    def val_dataloader(self):
        # OPTIONAL
        return DataLoader(DataSet_lung('valid'), batch_size=2)
        ##return DataLoader(MNIST(os.getcwd(), train=True, download=True, transform=transforms.ToTensor()), batch_size=32)

    def test_dataloader(self):
        # OPTIONAL
        return DataLoader(DataSet_lung('valid'), batch_size=2)
        #return DataLoader(MNIST(os.getcwd(), train=False, download=True, transform=transforms.ToTensor()), batch_size=32)
        
        
brain_model = BrainModel()

# most basic trainer, uses good defaults (1 gpu)
trainer = pl.Trainer(gpus=1,
                     max_epochs=2,
                     weights_summary=None)    
trainer.fit(brain_model)  



2020-05-22 18:41:59,552 distrib_data_parallel.py[220] INFO GPU available: True, used: True
2020-05-22 18:41:59,553 distrib_data_parallel.py[268] INFO VISIBLE GPUS: 0


False    6334
True     1583
Name: valid, dtype: int64


{'val_loss': tensor(0.5130, device='cuda:0')}


False    6334
True     1583
Name: valid, dtype: int64
False    6334
True     1583
Name: valid, dtype: int64


{'val_loss': tensor(0.0706, device='cuda:0')}


{'val_loss': tensor(0.0389, device='cuda:0')}



1